<a href="https://colab.research.google.com/github/Youssef-Rachad/QOSF-submission/blob/main/random_circuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%script
pip install qiskit

In [ ]:
from qiskit import QuantumCircuit, transpile, QuantumRegister
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
import numpy as np
from random import randint, seed, sample, choice, choices
import time

## Basic Version

In [ ]:
from qiskit.circuit.library import (IGate, U1Gate, U2Gate, U3Gate, XGate,
                                    YGate, ZGate, HGate, SGate, SdgGate, TGate,
                                    TdgGate, RXGate, RYGate, RZGate, CXGate,
                                    CYGate, CZGate, CHGate, CRZGate, CU1Gate,
                                    CU3Gate, SwapGate, RZZGate,
                                    CCXGate, CSwapGate)
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit
def random_circuit (num_qubits:int, depth:int, basis_gates:list, measure:bool, seed:int = None):
    '''
    num_qubits : integer value that is the number of qubits.
    depth: integer value that is the depth for the random circuit.
    basis_gates : A list that contains the basis gates to generate the quantum circuit.
    measure: boolean indicating whether values should be measured and reported into a classical register
    seed: integer value specifying seed for reproducible results and easier testing
    Return the quantum circuit
    '''
    # Setting the seed, if not specified then use current time
    if seed:
        np.random.seed(seed)
    else:
        np.random.seed(int(time.time()))
    # Dictionary to cross check basis gates
    # Keys represent number of qubit inputs
    gates = {"1": [IGate, U1Gate, U2Gate, U3Gate, XGate, YGate, ZGate, HGate, SGate, SdgGate, TGate, TdgGate, RXGate, RYGate, RZGate],
             "2": [CXGate, CYGate, CZGate, CHGate, CRZGate, CYGate, CU3Gate, SwapGate, RZZGate],
             "3": [CCXGate, CSwapGate]
           }
    # Dictionnary to cross check number of parameters
    parameters = {"1": [U1Gate, RXGate, RYGate, RZGate, RZZGate, CU1Gate, CRZGate],
                  "2": [U2Gate],
                  "3": [U3Gate, CU3Gate]
                 }
    # Take the union of the of the basis_gates and the defined gates
    # See of their lengths are greater than one to confirm gates of that size exist in the basis
    # Take the maximum allowed qubit input count
    max_count = max(
        (len(list(set(basis_gates)&set(gates["1"])))>0)*1,
        (len(list(set(basis_gates)&set(gates["2"])))>0)*2,
        (len(list(set(basis_gates)&set(gates["3"])))>0)*3,
    )
    # Define Quantum Register, and Circuit
    quantum_register = QuantumRegister(num_qubits, 'q')
    circuit = QuantumCircuit(quantum_register)
    # If measure is set True, set a classical register
    if measure:
            classical_register = ClassicalRegister(num_qubits, 'c')
            circuit.add_register(classical_register)
    
    # Initialise angle parameters
    angles = []
    # For each layer of depth, going horiztntally
    for layer in range(depth):
        # initialise with number of qubits
        remaining = list(range(num_qubits))
        # while there are qubits remaining
        while len(remaining):
            
            if len(remaining) > 1:
                # Take the minimum between the maximum qubit input allowed and the remaining length
                minimum = min(max_count, len(remaining))
                
                possible_qubits = range(0,minimum)
                weighting = [0.8, 0.1, 0.5, 0.3][:minimum] # make certain gates more likely
                number_of_operands = choices(possible_qubits, k=1, weights=weighting)[0]
                
                if number_of_operands == 0: # catch qubit not being operated on
                    pop = sample(range(len(remaining)), k=1)
                    remaining.pop(*pop)
                    continue
                pop = sample(range(len(remaining)), k=number_of_operands)
            else: # catch 1 qubit case (did not play nicely with following code)
                number_of_operands = 1
                pop = [0]
            
            pop.sort(reverse=True) # else index mismatch, descending doesnt affect later indices
            qubits = list(map(lambda x: remaining.pop(x), pop)) # remove qubits operated on for this layer
            
            if number_of_operands == 1:
                valid_gates = gates["1"]
            elif number_of_operands == 2:
                valid_gates = gate["2"]
            elif number_of_operands == 3:
                valid_gates = gates["3"]
            
            gate = choice(list(x for x in basis_gates if x in valid_gates))
            # Set angle parameter values
            for i in parameters:
                if gate in parameters[i]:
                    angles = list(map(lambda x:np.random.uniform(low=0, high=2*np.pi), list(range(int(i)))))
                    break
                else:
                    angles = []
            # Construct layer
            gate = gate(*angles)
            circuit.append(gate, [quantum_register[qubit] for qubit in qubits])
            
    if measure:
        circuit.measure(quantum_register, classical_register)
            
    return circuit


## Improved version
Going beyond the task prompt, I was able to make some modifications that would produce a nicer and more functional code.

In [ ]:
from qiskit.circuit.library import (IGate, U1Gate, U2Gate, U3Gate, XGate,
                                    YGate, ZGate, HGate, SGate, SdgGate, TGate,
                                    TdgGate, RXGate, RYGate, RZGate, CXGate,
                                    CYGate, CZGate, CHGate, CRZGate, CU1Gate,
                                    CU3Gate, SwapGate, RZZGate,
                                    CCXGate, CSwapGate)
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit
def random_circuit (num_qubits:int, depth:int, basis_gates:list, measure:bool, seed:int = None):
    '''
    num_qubits : integer value that is the number of qubits.
    depth: integer value that is the depth for the random circuit.
    basis_gates : A list that contains the basis gates to generate the quantum circuit.
    measure: boolean indicating whether values should be measured and reported into a classical register
    seed: integer value specifying seed for reproducible results and easier testing
    Return the quantum circuit
    '''
    # Setting the seed, if not specified then use current time
    if seed:
        np.random.seed(seed)
    else:
        np.random.seed(int(time.time()))
    # Dictionary to cross check basis gates
    # Keys represent number of qubit inputs
    gates = {"1": [IGate, U1Gate, U2Gate, U3Gate, XGate, YGate, ZGate, HGate, SGate, SdgGate, TGate, TdgGate, RXGate, RYGate, RZGate],
             "2": [CXGate, CYGate, CZGate, CHGate, CRZGate, CYGate, CU3Gate, SwapGate, RZZGate],
             "3": [CCXGate, CSwapGate]
           }
    # Dictionnary to cross check number of parameters
    parameters = {"1": [U1Gate, RXGate, RYGate, RZGate, RZZGate, CU1Gate, CRZGate],
                  "2": [U2Gate],
                  "3": [U3Gate, CU3Gate]
                 }
    # Take the union of the of the basis_gates and the defined gates
    # See of their lengths are greater than one to confirm gates of that size exist in the basis
    # Take the maximum allowed qubit input count
    max_count = max(
        (len(list(set(basis_gates)&set(gates["1"])))>0)*1,
        (len(list(set(basis_gates)&set(gates["2"])))>0)*2,
        (len(list(set(basis_gates)&set(gates["3"])))>0)*3,
    )
    # Define Quantum Register, and Circuit
    quantum_register = QuantumRegister(num_qubits, 'q')
    circuit = QuantumCircuit(quantum_register)
    # If measure is set True, set a classical register
    if measure:
            classical_register = ClassicalRegister(num_qubits, 'c')
            circuit.add_register(classical_register)
    
    # Initialise angle parameters
    angles = []
    # For each layer of depth, going horiztntally
    for layer in range(depth):
        # initialise with number of qubits
        remaining = list(range(num_qubits))
        # while there are qubits remaining
        while len(remaining):
            
            if len(remaining) > 1:
                # Take the minimum between the maximum qubit input allowed and the remaining length
                minimum = min(max_count, len(remaining))
                
                possible_qubits = range(0,minimum)
                weighting = [0.8, 0.1, 0.5, 0.3][:minimum] # make certain gates more likely
                number_of_operands = choices(possible_qubits, k=1, weights=weighting)[0]
                
                if number_of_operands == 0: # catch qubit not being operated on
                    pop = sample(range(len(remaining)), k=1)
                    remaining.pop(*pop)
                    continue
                pop = sample(range(len(remaining)), k=number_of_operands)
            else: # catch 1 qubit case (did not play nicely with following code)
                number_of_operands = 1
                pop = [0]
            
            pop.sort(reverse=True) # else index mismatch, descending doesnt affect later indices
            qubits = list(map(lambda x: remaining.pop(x), pop)) # remove qubits operated on for this layer
            
            if number_of_operands == 1:
                valid_gates = gates["1"]
            elif number_of_operands == 2:
                valid_gates = gate["2"]
            elif number_of_operands == 3:
                valid_gates = gates["3"]
            
            gate = choice(list(x for x in basis_gates if x in valid_gates))
            # Set angle parameter values
            for i in parameters:
                if gate in parameters[i]:
                    angles = list(map(lambda x:np.random.uniform(low=0, high=2*np.pi), list(range(int(i)))))
                    break
                else:
                    angles = []
            # Construct layer
            gate = gate(*angles)
            circuit.append(gate, [quantum_register[qubit] for qubit in qubits])
            
    if measure:
        circuit.measure(quantum_register, classical_register)
            
    return circuit
